# Projeto 1 - Ciência dos Dados

Nome: Filippo Ferraro

Nome: Nicolas Byung Kwan Cho

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [14]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import emoji

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [15]:
import os

filename = 'iPad Pro.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo iPad Pro.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [16]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,novo ipad air destruidor demais. não precisa g...,1
1,"@beniiciojr @elizabethrsic ih, facil é só eu p...",0
2,o negócio é comprar o ipad air preto e dizer q...,1
3,"vídeo feito com realidade aumentada, alguma su...",1
4,"@drudu mas migo, n é meio parecido com o últim...",1


In [17]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,rt @guiqis: novo ipad air: layout do ipad pro ...,1
1,novas cores do novo airpad air.\n\n- novo proc...,1
2,eu só quero um ipad pro,1
3,usb-c estará presente no novo ipad air.\n\num ...,1
4,"o design é baseado no ipad pro de 2018, soment...",1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O iPad Pro é um dispositivo do tipo tablet desenvolvido pela Apple. Por ser um produto de uma marca renomada e ter alto desempeno, o iPad Pro apresenta um valor elevado e não é um dispositivo acessível a todos.
Portanto, tweets que mencionavam o preço do produto, design, utilidade e especificações técnicas, foram considerados relevantes.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [18]:
import re 

def limpador(texto):

    pontuacao = '[!-.:?;@\n@_“”]'
    pattern = re.compile(pontuacao)
    subs = re.sub(pattern, '', texto)
    
    #separa palavra de emoji e emoji de emoji
    a = False
    for e in subs:
        if e in emoji.UNICODE_EMOJI:
            limpo = subs.replace(e, ' ' + e + ' ')
            a = True
        
    
    if a == True:
        return limpo
    else:
        return subs

In [19]:
#limpando os tweets
train.Treinamento.astype(str)
train.Treinamento = train.Treinamento.apply(limpador)

In [20]:
#colocando as palavras dos tweets relevantes em uma serie pd
tweets_relevantes = train.loc[train.Relevância == 1, : ]
todos_tweets_relevantes = ''
for e in tweets_relevantes.Treinamento:
    todos_tweets_relevantes += e

lista_tweets_relevantes = todos_tweets_relevantes.split()
series_tweets_relevantes = pd.Series(lista_tweets_relevantes)

In [21]:
#suavizacao de laplace
palavras_desconhecidas = 1000000
bonus = 0.5

In [22]:
#frequencias relativas das palavras em tweets relevantes (com suavizacao de laplace)
freq_tweets_relevantes = series_tweets_relevantes.value_counts() 
total_relevantes = sum(freq_tweets_relevantes) + palavras_desconhecidas*bonus

freq_rel_tweets_relevantes = (freq_tweets_relevantes+bonus) / total_relevantes

In [23]:
#colocando as palavras dos tweets irrelevantes em uma serie pd
tweets_irrelevantes = train.loc[train.Relevância == 0, : ]
todos_tweets_irrelevantes = ''
for e in tweets_irrelevantes.Treinamento:
    todos_tweets_irrelevantes += e

lista_tweets_irrelevantes = todos_tweets_irrelevantes.split()
series_tweets_irrelevantes = pd.Series(lista_tweets_irrelevantes)

In [24]:
#frequencias relativas das palavras em tweets irrelevantes (com suavizacao de laplace)
freq_tweets_irrelevantes = series_tweets_irrelevantes.value_counts() 
total_irrelevantes = sum(freq_tweets_irrelevantes) + palavras_desconhecidas*bonus

freq_rel_tweets_irrelevantes = (freq_tweets_irrelevantes+bonus) / total_irrelevantes

In [25]:
#desenvolvendo o classificador
def classificador(tweet):
    lista_palavras = tweet.split()
    
    P_tweet_dado_relevante = 1
    P_tweet_dado_irrelevante = 1
    
    for palavra in lista_palavras:
        if palavra in freq_rel_tweets_relevantes:
            P_tweet_dado_relevante *= freq_rel_tweets_relevantes[palavra]
        else:
            P_tweet_dado_relevante *= bonus/total_relevantes
            
        if palavra in freq_rel_tweets_irrelevantes:
            P_tweet_dado_irrelevante *= freq_rel_tweets_irrelevantes[palavra]
        else:
            P_tweet_dado_irrelevante *= bonus/total_irrelevantes
            
    P_relevante = len(tweets_relevantes)/len(train)
    P_irrelevante = len(tweets_irrelevantes)/len(train)
    
    P_relevante_dado_tweet = P_tweet_dado_relevante * P_relevante
    P_irrelevante_dado_tweet = P_tweet_dado_irrelevante * P_irrelevante
    
    
    
    if P_relevante_dado_tweet > P_irrelevante_dado_tweet:
        diferenca = (P_relevante_dado_tweet-P_irrelevante_dado_tweet)
        if diferenca > 10e-60:
            return 5
        elif diferenca <10e-200:
            return 3
        else:
            return 4
    else:
        diferenca = (P_irrelevante_dado_tweet-P_relevante_dado_tweet)
        if diferenca > 10e-60:
            return 1
        elif diferenca <10e-200:
            return 3
        else:
            return 2

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [26]:
#limpando os tweets
test.Teste.astype(str)
test.Teste = test.Teste.apply(limpador)
test.head(5)

,Teste,Relevância
0,rt guiqis novo ipad air layout do ipad pro com...,1
1,novas cores do novo airpad air novo processado...,1
2,eu só quero um ipad pro,1
3,usbc estará presente no novo ipad airum ipad m...,1
4,o design é baseado no ipad pro de 2018 somente...,1


In [27]:
lista_classificador = []
for tweet in test.Teste:
    lista_classificador.append(classificador(tweet))
    
series_classificador = pd.Series(lista_classificador)

In [28]:
test['Classificador'] = series_classificador
test['Classificador'] = test['Classificador'].astype('category')
test['Classificador'].cat.categories = ['muito irrelevante','irrelevante','neutro','relevante','muito relevante']

In [29]:
comparacao = pd.crosstab(test.Relevância, test.Classificador, normalize = 'all')
comparacao

Classificador,muito irrelevante,irrelevante,neutro,relevante,muito relevante
Relevância,,,,,
0,0.029851,0.134328,0.067164,0.149254,0.044776
1,0.014925,0.014925,0.052239,0.350746,0.141791


In [30]:
verdadeiros_positivos = comparacao.iloc[1]['relevante'] + comparacao.iloc[1]['muito relevante']
falsos_positivos = comparacao.iloc[0]['relevante'] + comparacao.iloc[0]['muito relevante']
verdadeiros_negativos = comparacao.iloc[0]['muito irrelevante'] + comparacao.iloc[0]['irrelevante']
falsos_negativos = comparacao.iloc[1]['muito irrelevante'] + comparacao.iloc[1]['irrelevante']
neutros = comparacao.iloc[0]['neutro'] + comparacao.iloc[1]['neutro']

print('Verdadeiros positivos:', verdadeiros_positivos)
print('Falsos positivos:', falsos_positivos)
print('Verdadeiros negativos:', verdadeiros_negativos)
print('Falsos_negativos:', falsos_negativos)
print('Neutros:', neutros)

Verdadeiros positivos: 0.4925373134328358
Falsos positivos: 0.19402985074626863
Verdadeiros negativos: 0.16417910447761194
Falsos_negativos: 0.029850746268656716
Neutros: 0.11940298507462686


In [31]:
acertos = verdadeiros_negativos + verdadeiros_positivos
erros = falsos_negativos + falsos_positivos
print('Acertos:', acertos)
print('Erros:', erros)
print('Neutros:', neutros)

Acertos: 0.6567164179104478
Erros: 0.22388059701492535
Neutros: 0.11940298507462686


___
### Concluindo

-Uma parte considerável dos tweets com relevância 0 foi classificada como relevante ou muito relevante, já os tweets com relevância 1 seguiram de acordo com o esperado e apenas uma pequena porcentagem foi classificada como muito irrelevante. Mas, observando a performance do classificador, temos uma taxa de erro de apenas 22,39%.

Alguns fatores que podem ter dificultado a classificação:
* O fato do produto apresentar a palavra 'pro' no nome, pode ter afetado o desempenho do classificador por confundir com a junção da preposição 'para' com o pronome 'o' ('para'+'o' = 'pro').
* Outros produtos da Apple possuem 'pro' no nome (como 'iPhone Pro').
* Erros de ortografia nos tweeets.

-Mensagens que apresentam sarcasmo ou dupla negação são entendidas pelo classificador como o sentido literal das palavras e não seu oposto. Assim, tweets que desejam expressar algo irrelevante mas utiliza palavras relevantes são classificados como relevantes.

-Este trabalho deve continuar sendo financiado pois, por enquanto, o classificador está funcionando apenas com uma base de treinamento previamente classificada e já apresenta bons resultados. Visto isso, planejamos aperfeiçoar e preparar o projeto para que seja capaz de alimentar sua base automaticamente usando o próprio classificador, aplicado a novos tweets (trabalhar com dados não supervisionados).

#### Complementando

-O classificador Naïve-Bayes é uma ferramenta poderosa considerando sua simplicidade, mas possui limitações, principalmente pelo fato de considerar independência entre os eventos, sendo bem improvável no mundo real. Contudo, é um bom algoritmo para as seguintes aplicações: 
* Classificador de emails: spam ou não spam
* Classificador de sexo por meio de características pessoais (tamanho, peso, etc...)
* Atribuir diagnósticos médicos baseados em sintomas

-Nao se pode utilizar o próprio classificador para gerar mais amostras de treinamento, pois a precisão do classificador não é perfeita, podendo classificar dados de forma errônea, alimentando a própria base de treinamento com informações erradas e comprometendo o classificador.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**